# Entrenamiento de los agentes basados en redes neuronales

Para entrenar a los agentes, deberemos definir un set de datos con estados del juego y un set de etiquetas con la acción preferida (representada en un vector de largo 5 con ceros en todas las posiciones excepto la de la acción elegida).

Luego, entrenaremos la red neuronal en este set de datos y la guardaremos en un archivo (``NN[Agent].h5``), el cual podrá ser accedido por el agente en el archivo ``neural.py``.

In [19]:
# Librerías utilizadas para trabajar en el notebook
import tensorflow as tf # Instalar mediante pip install tensorflow
import numpy as np
import random
import os

In [20]:
# Funciones auxiliares (que pueden resultar útiles) y rival de baseline, entregado con la tarea
from agents.baseline import BaseCat, BaseMouse
from utils import bfs_search, get_valid_moves

In [21]:
# Cargamos el mapa de juego, para poder buscar espacios libres dentro de él
lab_map = np.load(os.path.join("game_map.npy"))

In [22]:
# Obtenemos los espacios libres dentro del mapa
free_positions = []

for x in range(lab_map.shape[0]):
    for y in range(lab_map.shape[1]):
        if lab_map[x, y] == 0:
            free_positions.append((x, y))

free_positions[:5]

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]

## Entrenamiento del gato 🐈

### Creación del set de datos

Generaremos un set de datos ``X`` que contendrá numpy arrays con la posición del gato y el ratón ``(cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1])``.

Además, generaremos un set de etiquetas ``y`` que contendrá numpy arrays con el movimiento a realizar (ej. ``(0, 0, 1, 0, 0)`` significa hacer el movimiento 2).

La forma en que calculas qué acción llevar a cabo dado el estado de juego es tu decisión, puede ser una búsqueda utilizando el algoritmo de BFS entregado, una política manual o una serie de reglas if-else. Lo importante es que este será el comportamiento que la red neuronal del agente tratará de aproximar. Recuerda que dispones del mapa de juego en ``lab_map`` y de un agente contra el que entrenar bajo la clase `BaseMouse`.

In [23]:
# Instanciamos al jugador oponente
player = BaseCat((0, 0))

# Trabajaremos a X e y como listas, en la siguiente celda las transformaremos a un array de numpy
# (los componentes que colocas en ellas si deben ser numpy arrays)
X = []
y = []

# Definir los datos de entrada
acciones = [0, 1, 2, 3, 4]
# Crear una matriz de ceros con la forma adecuada
label = np.zeros((len(acciones), len(acciones)), dtype=int)
# Rellenar la matriz one-hot
for i, accion in enumerate(acciones):
    label[i, accion] = 1

# Para cada combinación de posiciones de gato y ratón
for cat_pos in free_positions:
    for mouse_pos in free_positions:
        X.append(np.array([cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1]]))
        
        # Obtener la acción del rival
        mouse_pos_array = np.array(mouse_pos)
        cat_pos_array = np.array(cat_pos)
        move = player.get_action(lab_map, cat_pos_array, mouse_pos_array)

        # Asignar movimiento y añadirlo a la lista de etiquetas
        y.append(label[move])

In [24]:
# Transformamos las listas a numpy arrays
X = np.array(X)
y = np.array(y)

In [25]:
print(X.shape)
print(y.shape)

(6561, 4)
(6561, 5)


### Diseño y entrenamiento de la Red Neuronal

Para entrenar nuestra red neuronal, utilizaremos la librería de [`TensorFlow`](https://www.tensorflow.org/api_docs/python/tf/all_symbols), podemos instalar esta librería mediante la línea `pip install tensorflow`.

TensorFlow es una librería desarrollada por Google que nos permite construir, entrenar e implementar modelos de aprendizaje profundo en Python.

Para esto, TensorFlow nos permite armar una red en forma de "capas", [en el siguiente link encontrarás un tutorial más en detalle de cómo crear un modelo de TensorFlow](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5). Deberemos introducir todas las capas de nuestra red dentro de un objeto `tf.keras.Sequential()`, que recibe de parámetro una lista con todos los elementos de nuestra red.

<br>

A continuación se enumeran los tres principales elementos que utilizaremos en nuestro modelo:
- `tf.keras.Input(shape = a)`: La capa de entrada de la red, es la primera que recibe. Necesita de un parámetro `shape` que determinará la dimensionalidad del vector de entrada, esto será entregado en una tupla `a` (ej. `(5, 2)`).

- `tf.keras.layers.Dense(units = b, activation = 'relu')`: Corresponde a una capa intermedia de la red del tipo *Fully Connected*, le entregaremos un parámetro `units` que determinará cuántas unidades ocultas (neuronas) tendrá la capa (y por tanto, el número de componentes en el vector de salida de la capa, en este caso `b`). Además, recibirá una función de activación para cada neurona, esta puede ser del tipo `relu`, `tanh`, `softmax` u otras, para más detalle sobre ellas visitar [el siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/activations). Recomendamos usar `relu` para capas intermedias.

- `tf.keras.layers.Dense(units = c, activation = 'softmax')`: Similar a lo explicado anteriormente, corresponde a una capa densa de activación `softmax`. Esta corresponderá a la capa final del modelo, con `c` el número de elementos que deseamos que nuestro vector de salida tenga. El uso de una función softmax se debe a que esta nos permite llevar la salida de nuestra neurona a un conjunto de probabilidades normalizadas, desde el cual podemos calcular qué acción jugar. [En el siguiente link](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer) se explica el funcionamiento de esta función en mayor detalle.

<br>

Un ejemplo de una red simple sería:
```
model = tf.keras.Sequential([
    tf.keras.Input(shape = a),
    tf.keras.layers.Dense(b, activation = "relu"),
    tf.keras.layers.Dense(c, activation = "softmax")
])
```

In [26]:
# Recuerda que para esta tarea la entrada es un vector de 4 elementos y buscamos una salida de 5 elementos (la probabilidad de jugar acción posible)

# ===== COMPLETAR =====
# Se debe crear un modelo para nuestra red neuronal, añade cuantas capas ocultas desees, con cuantas neuronas desees
model = tf.keras.Sequential([
    tf.keras.Input(shape=(4,)),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
# =====================

TensorFlow nos permite además compilar un modelo, especificando qué optimizador utilizar para su ajuste de parámetros y qué función de pérdida utilizar para hacer la backpropagation.

Compilaremos un modelo de la forma:
``model.compile(optimizer = 'optimizer_name', loss = 'loss_name', metrics = ['accuracy'])``

- Con `optimizer_name` el nombre de algún optimizador (por ejemplo, `sgd` o `adam`, más optimizadores en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers))
- Con `loss_name` el nombre de alguna función de pérdida (por ejemplo, `binary_crossentropy`, más funciones en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/losses))
- Usaremos de métrica el accuracy del modelo, aunque existe [múltiples otras métricas que incluir](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [27]:
# ===== COMPLETAR =====
# Compila tu modelo de TensorFlow
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
# =====================


Ahora que especificamos cómo vamos calcular el error del modelo y cómo lo usaremos para ajustar sus pesos, es hora de entrenar nuestra red. Para ello, utilizaremos su método `.fit()` de la siguiente forma:

`model.fit(X, y, epochs = a, batch_size = b)`

- `X` corresponde a nuestra matriz de características
- `y` corresponde a nuestro vector de etiquetas para cada muestra
- `epochs` determina cuántas veces iteraremos sobre todo el set de datos
- `batch_size` determina cuántas muestras serán consideradas por propagación del gradiente (un valor mayor significa entrenamiento más rápido pero mayor uso de memoria)

In [28]:
# ===== COMPLETAR =====
# Entrena tu modelo de TensorFlow utilizando su método .fit()
model.fit(X, y, epochs=300, batch_size=500)
# =====================

Epoch 1/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3976 - loss: 1.4121
Epoch 2/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5244 - loss: 1.0721
Epoch 3/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5257 - loss: 1.0470
Epoch 4/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5382 - loss: 1.0242
Epoch 5/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5641 - loss: 0.9855
Epoch 6/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5593 - loss: 0.9830
Epoch 7/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5543 - loss: 0.9787
Epoch 8/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5807 - loss: 0.9428
Epoch 9/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5764 - loss: 0.9270
Epoch 10/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5850 - loss: 0.9304
Epoch 11/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5925 - loss: 0.9075
Epoch 12/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy:

Por último, guardaremos el modelo entrenado en el directorio `agents/data/NNCat.h5` para poder acceder a este modelo desde nuestra clase `NNCat`:

In [29]:
model.save(os.path.join('agents', 'data', 'NNCat.h5'))

## Entrenamiento del ratón 🐁

### Creación del set de datos

Generaremos un set de datos ``X`` que contendrá numpy arrays con la posición del gato y el ratón ``(cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1])``.

Además, generaremos un set de etiquetas ``y`` que contendrá numpy arrays con el movimiento a realizar (ej. ``(1, 0, 0, 0, 0)`` significa hacer el movimiento 0).

La forma en que calculas qué acción llevar a cabo dado el estado de juego es tu decisión. Lo importante es que este será el comportamiento que la red neuronal del agente tratará de aproximar. Recuerda que dispones del mapa de juego en ``lab_map`` y de un agente contra el que entrenar bajo la clase `BaseCat`.

In [30]:
# Instanciamos al jugador oponente
player = BaseMouse((0, 0))

X = []
y = []

# Definir los datos de entrada
acciones = [0, 1, 2, 3, 4]
# Crear una matriz de ceros con la forma adecuada
label = np.zeros((len(acciones), len(acciones)), dtype=int)
# Rellenar la matriz one-hot
for i, accion in enumerate(acciones):
    label[i, accion] = 1

for cat_pos in free_positions:
    for mouse_pos in free_positions:
        X.append(np.array([cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1]]))

        # Obtener la acción del raton
        mouse_pos_array = np.array(mouse_pos)
        cat_pos_array = np.array(cat_pos)
       
        move = player.get_action(lab_map, cat_pos_array, mouse_pos_array)
        
        # Asignar movimiento y añadirlo a la lista de etiquetas
        y.append(label[move])

In [31]:
# Transformamos las listas a numpy arrays
X = np.array(X)
y = np.array(y)

### Diseño y entrenamiento de la Red Neuronal

Para entrenar nuestra red neuronal, utilizaremos la librería de [`TensorFlow`](https://www.tensorflow.org/api_docs/python/tf/all_symbols), podemos instalar esta librería mediante la línea `pip install tensorflow`.

TensorFlow es una librería desarrollada por Google que nos permite construir, entrenar e implementar modelos de aprendizaje profundo en Python.

Para esto, TensorFlow nos permite armar una red en forma de "capas", [en el siguiente link encontrarás un tutorial más en detalle de cómo crear un modelo de TensorFlow](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5). Deberemos introducir todas las capas de nuestra red dentro de un objeto `tf.keras.Sequential()`, que recibe de parámetro una lista con todos los elementos de nuestra red.

<br>

A continuación se enumeran los tres principales elementos que utilizaremos en nuestro modelo:
- `tf.keras.Input(shape = a)`: La capa de entrada de la red, es la primera que recibe. Necesita de un parámetro `shape` que determinará la dimensionalidad del vector de entrada, esto será entregado en una tupla `a` (ej. `(5, 2)`).

- `tf.keras.layers.Dense(units = b, activation = 'relu')`: Corresponde a una capa intermedia de la red del tipo *Fully Connected*, le entregaremos un parámetro `units` que determinará cuántas unidades ocultas (neuronas) tendrá la capa (y por tanto, el número de componentes en el vector de salida de la capa, en este caso `b`). Además, recibirá una función de activación para cada neurona, esta puede ser del tipo `relu`, `tanh`, `softmax` u otras, para más detalle sobre ellas visitar [el siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/activations). Recomendamos usar `relu` para capas intermedias.

- `tf.keras.layers.Dense(units = c, activation = 'softmax')`: Similar a lo explicado anteriormente, corresponde a una capa densa de activación `softmax`. Esta corresponderá a la capa final del modelo, con `c` el número de elementos que deseamos que nuestro vector de salida tenga. El uso de una función softmax se debe a que esta nos permite llevar la salida de nuestra neurona a un conjunto de probabilidades normalizadas, desde el cual podemos calcular qué acción jugar. [En el siguiente link](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer) se explica el funcionamiento de esta función en mayor detalle.

<br>

Un ejemplo de una red simple sería:
```
model = tf.keras.Sequential([
    tf.keras.Input(shape = a),
    tf.keras.layers.Dense(b, activation = "relu"),
    tf.keras.layers.Dense(c, activation = "softmax")
])
```

In [32]:
# Recuerda que para esta tarea la entrada es un vector de 4 elementos y buscamos una salida de 5 elementos (la probabilidad de jugar acción posible)

# ===== COMPLETAR =====
# Se debe crear un modelo para nuestra red neuronal, añade cuantas capas ocultas desees, con cuantas neuronas desees
model = tf.keras.Sequential([
    tf.keras.Input(shape=(4,)),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
# =====================

TensorFlow nos permite además compilar un modelo, especificando qué optimizador utilizar para su ajuste de parámetros y qué función de pérdida utilizar para hacer la backpropagation.

Compilaremos un modelo de la forma:
``model.compile(optimizer = 'optimizer_name', loss = 'loss_name', metrics = ['accuracy'])``

- Con `optimizer_name` el nombre de algún optimizador (por ejemplo, `sgd` o `adam`, más optimizadores en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers))
- Con `loss_name` el nombre de alguna función de pérdida (por ejemplo, `binary_crossentropy`, más funciones en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/losses))
- Usaremos de métrica el accuracy del modelo, aunque existe [múltiples otras métricas que incluir](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [33]:
# ===== COMPLETAR =====
# Compila tu modelo de TensorFlow
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
# =====================

Ahora que especificamos cómo vamos calcular el error del modelo y cómo lo usaremos para ajustar sus pesos, es hora de entrenar nuestra red. Para ello, utilizaremos su método `.fit()` de la siguiente forma:

`model.fit(X, y, epochs = a, batch_size = b)`

- `X` corresponde a nuestra matriz de características
- `y` corresponde a nuestro vector de etiquetas para cada muestra
- `epochs` determina cuántas veces iteraremos sobre todo el set de datos
- `batch_size` determina cuántas muestras serán consideradas por propagación del gradiente (un valor mayor significa entrenamiento más rápido pero mayor uso de memoria)

In [34]:
# ===== COMPLETAR =====
# Entrena tu modelo de TensorFlow utilizando su método .fit()
model.fit(X, y, epochs=400, batch_size=500)
# =====================

Epoch 1/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3646 - loss: 1.4918
Epoch 2/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4409 - loss: 1.3467
Epoch 3/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4430 - loss: 1.2920
Epoch 4/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4775 - loss: 1.2527
Epoch 5/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4841 - loss: 1.2428
Epoch 6/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4678 - loss: 1.2165
Epoch 7/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4849 - loss: 1.1783
Epoch 8/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4845 - loss: 1.1873
Epoch 9/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4877 - loss: 1.1777
Epoch 10/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4997 - loss: 1.1431
Epoch 11/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4938 - loss: 1.1395
Epoch 12/400
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy

Por último, guardaremos el modelo entrenado en el directorio `agents/data/NNMouse.h5` para poder acceder a este modelo desde nuestra clase `NNMouse`:

In [35]:
model.save(os.path.join('agents', 'data', 'NNMouse.h5'))

### Análisis

**Explique por qué para esta implementación en particular es conveniente tener overfitting sobre los datos ¿Es este el caso para la mayoría de las aplicaciones?**

Tener overfitting sobre los datos puede ser conveniente en casos específicos donde el objetivo principal es maximizar el rendimiento en un conjunto de datos particular, en lugar de generalizar a nuevos datos. En este caso no nos interesa mucho generalizar y al minuto de tener overfitting sobre los datos, la red neuronal se ajusta de manera muy precisa a los ejemplos de entrenamiento que serían la acción tal que los jugadores se acerquen o alejen segun corresponda y aprenderá a realizar movimientos específicos que maximizan su rendimiento en ese conjunto de datos particular.

**Indica dos contraejemplos en donde no deseemos tener overfitting sobre nuestros datos. (Pista: Investiga sobre el concepto de generalización)**

- *Clasificación de imágenes:* Si queremos un modelo que nos clasifique imágenes no podemos tener overfitting ya que esto significaría que no podría generalizar y se ajustaría significativamente a los datos de entrenamiento. Si el modelo se ajusta mucho, puede memorizar las características específicas de esas imágenes y perder la capacidad de generalizar correctamente. Por ejemplo, puede aprender detalles insignificantes o ruido presente en las imágenes de entrenamiento, lo que lleva a una pérdida de su capacidad para clasificar correctamente nuevas imágenes.

- *Reconocimiento de voz*: Si permitimos que se produzca overfitting durante el entrenamiento de este modelo, significa que este se ajustará demasiado a las características y voces específicas de los hablantes en el conjunto de entrenamiento. Como resultado, el modelo podría tener dificultades para generalizar correctamente a hablantes nuevos y diferentes que no se encuentren en el conjunto de entrenamiento. El overfitting en el reconocimiento de voz puede llevar a errores de transcripción y resultados poco confiables si se busca transcribir a hablantes con diferentes acentos y entonaciones que pueden no haber sido parte del set de entrenamiento. 

**¿Por qué se utiliza la función de activación softmax en la salida de la red? Explica brevemente**

Se usa para clasificaciones multiclase. Como ahora queremos que el modelo nos indica que acción realizar, softmax toma un vector de valores reales y produce un nuevo vector de la dimensión de salida (en este caso 5 por las 5 acciones posibles) en el que cada acción está en el rango de 0 a 1 y la suma de todas es igual a 1. Es decir, que para cada acción entrega la probabilidad de que esa acción sea la más eficiente.

**¿Cuántas capas contiene tu modelo? ¿Cuántas neuronas contiene cada capa? Se espera que expliques tu razonamiento y/o proceso para llegar a esos números**

3 capas ocultas con 300,200 y 100 neuronas respectivamente. Como no es un problema complejo y no existen muchos datos estime que 3 capas y la cantidad de neuronas sería suficiente para predecir la acción ideal. También experimenté con estos números de manera de tener las mejores eficiencias sin requerir mucho cómputo.